In [1]:
from IPython.display import HTML

In [2]:
import requests


In [3]:
res = requests.get('http://project.initiumlab.com/news2video/case1/index.html')

In [4]:
html = res.content.decode('utf-8')

In [5]:
from readability.readability import Document
readable_article = Document(html).summary()
readable_title = Document(html).short_title()

In [6]:
readable_article

'<html><body><div><article id="main">\n\n  <h1>Walk: A Boring Story</h1>\n\n  I\'m going to tell you a boring story.\n\n  <img src="1.jpg" alt=""/>\n  <p>\n    This is the first step I move.\n  </p>\n\n  <img src="2.jpg" alt=""/>\n  <p>\n    This is the second step I move.\n  </p>\n\n  <img src="3.jpg" alt=""/>\n  <p>\n    This is the third step I move.\n  </p>\n\n  <img src="4.jpg" alt=""/>\n  <p>\n    This is the fourth step I move.\n  </p>\n\n  <img src="5.jpg" alt=""/>\n  <p>\n    This is the fifth step I move.\n  </p>\n\n  <img src="6.jpg" alt=""/>\n  <p>\n    This is the sixth step I move.\n  </p>\n\n  <img src="7.jpg" alt=""/>\n  <p>\n    This is the seventh step I move.\n  </p>\n\n  <img src="8.jpg" alt=""/>\n  <p>\n    This is the eighth step I move.\n  </p>\n\n  <img src="9.jpg" alt=""/>\n  <p>\n    This is the final step I move.\n  </p>\n\n  <p>\n    Thank you for reading this story.\n  </p>\n\n</article>\n\n</div></body></html>'

In [7]:
HTML(readable_article)

In [8]:
from os import path
base_url = path.dirname(res.request.url)
print(res.request.url)
print(base_url)

http://project.initiumlab.com/news2video/case1/index.html
http://project.initiumlab.com/news2video/case1


In [9]:
%%file test.txt
test

Overwriting test.txt


In [10]:
import html2text
print(html2text.html2text(readable_article))

# Walk: A Boring Story

I'm going to tell you a boring story. ![](1.jpg)

This is the first step I move.

![](2.jpg)

This is the second step I move.

![](3.jpg)

This is the third step I move.

![](4.jpg)

This is the fourth step I move.

![](5.jpg)

This is the fifth step I move.

![](6.jpg)

This is the sixth step I move.

![](7.jpg)

This is the seventh step I move.

![](8.jpg)

This is the eighth step I move.

![](9.jpg)

This is the final step I move.

Thank you for reading this story.




In [11]:
from pyquery import PyQuery as pq

In [12]:
#r = pq(readable_article)
#r('img').map(lambda i, e: pq(e).attr('src', '%s/%s' % (base_url, pq(e).attr('src'))))

In [13]:
#readable_article

In [14]:
from html.parser import HTMLParser
#from HTMLParser import HTMLParser
#from htmlentitydefs import name2codepoint

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)
        for attr in attrs:
            print("     attr:", attr)
    def handle_endtag(self, tag):
        print("End tag  :", tag)
    def handle_data(self, data):
        print("Data     :", data)
    def handle_comment(self, data):
        print("Comment  :", data)
    def handle_entityref(self, name):
        #c = unichr(name2codepoint[name])
        c = 'entity name should be here'
        print("Named ent:", c)
    def handle_charref(self, name):
        if name.startswith('x'):
            c = unichr(int(name[1:], 16))
        else:
            c = unichr(int(name))
        print("Num ent  :", c)
    def handle_decl(self, data):
        print("Decl     :", data)

parser = MyHTMLParser()

In [15]:
parser.feed(readable_article)

Start tag: html
Start tag: body
Start tag: div
Start tag: article
     attr: ('id', 'main')
Data     : 

  
Start tag: h1
Data     : Walk: A Boring Story
End tag  : h1
Data     : 

  I'm going to tell you a boring story.

  
Start tag: img
     attr: ('src', '1.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the first step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     attr: ('src', '2.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the second step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     attr: ('src', '3.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the third step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     attr: ('src', '4.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the fourth step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     a

Remark: not easy to use

In [21]:
# reference:
# http://stackoverflow.com/questions/20590624/python-beautifulsoup-div-text-and-img-attributes-in-correct-order
import bs4
from bs4 import Tag, BeautifulSoup 

class Extractor(object):
    def __init__(self):
        self.cur_text = ''
        self.result = []
    
    def recursive_extract_text_image(self, obj):
        for child in obj.children:
            if isinstance(child, Tag):
                #result.append(child.get('alt', ''))
                self.recursive_extract_text_image(child)
                if child.name == 'img':
                    self.result.append(('text', self.cur_text))
                    self.cur_text = ''
                    self.result.append(('image',
                                   '%s/%s' % (base_url, child['src'])
                                  ))
            else:
                if len(child.strip()) > 0:
                    self.cur_text += ' ' + child.strip() + ' '

    def html_to_asset_list(self, html):
        bs_obj = BeautifulSoup(html, 'html.parser')
        self.recursive_extract_text_image(bs_obj)
        self.result.append(('text', self.cur_text))
        return self.result

result = Extractor().html_to_asset_list(readable_article)
print(result)

[('text', " Walk: A Boring Story  I'm going to tell you a boring story. "), ('image', 'http://project.initiumlab.com/news2video/case1/1.jpg'), ('text', ' This is the first step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/2.jpg'), ('text', ' This is the second step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/3.jpg'), ('text', ' This is the third step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/4.jpg'), ('text', ' This is the fourth step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/5.jpg'), ('text', ' This is the fifth step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/6.jpg'), ('text', ' This is the sixth step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/7.jpg'), ('text', ' This is the seventh step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/8.jpg'), ('text', ' This is the eighth step I move. '), ('image', 'http

In [22]:
import pandas as pd

In [23]:
df_screenplay = pd.DataFrame(result, columns=['type', 'content'])

In [24]:
import hashlib 
def string2hash(s):
    m = hashlib.sha256()
    m.update(s.encode('utf-8'))
    return m.hexdigest()[:16]
df_screenplay['local_src'] = df_screenplay['content'].apply(lambda x: string2hash(x))

In [26]:
image_selector = (df_screenplay['type'] == 'image')
df_screenplay.loc[image_selector, 'filename'] = df_screenplay.loc[
    image_selector, 'content'].apply(lambda x: path.basename(x))
df_screenplay.loc[image_selector, 'extname'] = df_screenplay.loc[
    image_selector, 'filename'].apply(lambda x: path.splitext(x)[1])
df_screenplay

,type,content,local_src,filename,extname
0,text,Walk: A Boring Story I'm going to tell you a...,0b6f2772f708dd15,NaN,NaN
1,image,http://project.initiumlab.com/news2video/case1...,5bf67d1c0729bd50,1.jpg,.jpg
2,text,This is the first step I move.,e99d8b267c0f2b29,NaN,NaN
3,image,http://project.initiumlab.com/news2video/case1...,3a9e810ed3b010a4,2.jpg,.jpg
4,text,This is the second step I move.,07c025cc4e45e727,NaN,NaN
5,image,http://project.initiumlab.com/news2video/case1...,b30b2910be978147,3.jpg,.jpg
6,text,This is the third step I move.,b30a844bd7d16bd3,NaN,NaN
7,image,http://project.initiumlab.com/news2video/case1...,f7c4cc87ce36317c,4.jpg,.jpg
8,text,This is the fourth step I move.,f5329785a4ee9a50,NaN,NaN
9,image,http://project.initiumlab.com/news2video/case1...,bb32e45dfe24271c,5.jpg,.jpg
